In [1]:
import pandas as pd,re, pickle, scipy
from scipy.sparse import csr_matrix
from sklearn.naive_bayes import *
import numpy as np
from sklearn.feature_selection import *
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder as LE
from sklearn.preprocessing import *
from sklearn.decomposition import PCA
from sklearn.metrics import *
from sklearn.kernel_ridge import *
from sklearn import tree
from sklearn.ensemble import *
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.svm import LinearSVC, SVC, OneClassSVM
from sklearn.model_selection import *
from sklearn.linear_model import *
from collections import defaultdict

from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE, ADASYN
from url_utils import URLUtils
from keras.layers import *
from keras.models import  *
from keras.utils import to_categorical
from keras.metrics import *
from encoder import Encoder
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [3]:

def convert_val(val):
    return (val+1)/2

def parse_dates(date):
    parts=date.split(" ")
    date_parts = parts[0].split("-")
    time_parts = parts[1].split(':')
    new_date =date_parts[2]+date_parts[1]+date_parts[0]+time_parts[0]+time_parts[1]
    return int(new_date)

d = defaultdict(LE)
mms=MinMaxScaler()


In [ ]:

sample_path='/Users/bjayaram/Documents/Kaggle/talkingdata/mnt/ssd/kaggle-talkingdata2/competition_files/train_sample.csv'
path='/Users/bjayaram/Documents/Kaggle/talkingdata/mnt 2/ssd/kaggle-talkingdata2/competition_files/train.csv'

df = pd.read_csv(sample_path)
# df = df.loc[df['is_attributed']==1]
df['click_time']= df['click_time'].map(parse_dates)
data_cols = list(df.columns)
data_cols.remove('attributed_time')
data_cols.remove('is_attributed')
smote = SMOTE(ratio='minority',k_neighbors=15, n_jobs=-1)
label_cols = u'is_attributed'
print "Original Class dist:\n",df[label_cols].value_counts()
print 'Data generated'
y = df[label_cols].values
X = df[data_cols].values


[[  2.39976641e-01   2.00000000e-02   2.58598397e-04   1.50115473e-02
    9.97979798e-01   3.33333311e-01]
 [  2.89380613e-01   4.36363636e-02   2.58598397e-04   1.96304850e-02
    5.17171717e-01   3.33333325e-01]
 [  2.78041278e-01   2.00000000e-02   2.58598397e-04   2.19399538e-02
    4.22222222e-01   3.33333341e-01]
 ..., 
 [  7.70487392e-01   4.87162303e-02   8.82375656e-04   2.16208360e-02
    4.24242424e-01   6.66666660e-01]
 [  4.63837171e-01   4.52473514e-02   2.58598397e-04   2.22678081e-02
    1.23192342e-01   3.33333308e-01]
 [  1.30463668e-01   2.72269264e-02   7.82309407e-05   4.54127901e-03
    7.81732737e-01   9.99999973e-01]] [0 0 0 ..., 1 1 1]


In [11]:


X = mms.fit_transform(X)
x_train, y_train = smote.fit_sample(X, y)
x_train.shape,y_train.shape


df2=pd.DataFrame()

df2[label_cols]=y_train
print "After SMOTE Class dist:\n",df2[label_cols].value_counts()


[[  2.39976641e-01   2.00000000e-02   2.58598397e-04   1.50115473e-02
    9.97979798e-01   3.33333311e-01]
 [  2.89380613e-01   4.36363636e-02   2.58598397e-04   1.96304850e-02
    5.17171717e-01   3.33333325e-01]
 [  2.78041278e-01   2.00000000e-02   2.58598397e-04   2.19399538e-02
    4.22222222e-01   3.33333341e-01]
 ..., 
 [  7.70487392e-01   4.87162303e-02   8.82375656e-04   2.16208360e-02
    4.24242424e-01   6.66666660e-01]
 [  4.63837171e-01   4.52473514e-02   2.58598397e-04   2.22678081e-02
    1.23192342e-01   3.33333308e-01]
 [  1.30463668e-01   2.72269264e-02   7.82309407e-05   4.54127901e-03
    7.81732737e-01   9.99999973e-01]] [0 0 0 ..., 1 1 1]


In [13]:

print x_train, y_train

[[  2.39976641e-01   2.00000000e-02   2.58598397e-04   1.50115473e-02
    9.97979798e-01   3.33333311e-01]
 [  2.89380613e-01   4.36363636e-02   2.58598397e-04   1.96304850e-02
    5.17171717e-01   3.33333325e-01]
 [  2.78041278e-01   2.00000000e-02   2.58598397e-04   2.19399538e-02
    4.22222222e-01   3.33333341e-01]
 ..., 
 [  7.70487392e-01   4.87162303e-02   8.82375656e-04   2.16208360e-02
    4.24242424e-01   6.66666660e-01]
 [  4.63837171e-01   4.52473514e-02   2.58598397e-04   2.22678081e-02
    1.23192342e-01   3.33333308e-01]
 [  1.30463668e-01   2.72269264e-02   7.82309407e-05   4.54127901e-03
    7.81732737e-01   9.99999973e-01]] [0 0 0 ..., 1 1 1]


In [20]:

model = Sequential()
num_columns=x_train.shape[1]
model.add(Dense(num_columns, input_shape=(num_columns,), kernel_initializer='normal', activation='relu'))
model.add(Dense(50000, kernel_initializer='normal', activation='relu'))
# model.add(Dense(100, kernel_initializer='normal', activation='relu'))

model.add(Dense(2, kernel_initializer='normal', activation='softmax'))
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy',mae])


In [22]:


model.fit(x_train,to_categorical(y_train), verbose=3, validation_split=0.2, shuffle=True, epochs=7)
model.evaluate(x_train,to_categorical(y_train))


Train on 159636 samples, validate on 39910 samples
Epoch 1/7


Epoch 2/7


Epoch 3/7


Epoch 4/7


Epoch 5/7


...

198400/199546 [============================>.] - ETA: 0s

198912/199546 [============================>.] - ETA: 0s

199488/199546 [============================>.] - ETA: 0s

199546/199546 [==============================] - 19s 94us/step


[0.22277968821857683, 0.9101811111204553, 0.13449297137770164]

In [27]:

model.save('/Users/bjayaram/Documents/Kaggle/talkingdata/model_50000.h5')

Loaded model


Test data generated


In [28]:
model = load_model('/Users/bjayaram/Documents/Kaggle/talkingdata/model_50000.h5')
print 'Loaded model'
df = pd.read_csv('/Users/bjayaram/Documents/Kaggle/talkingdata/test.csv')
df['click_time']= df['click_time'].map(parse_dates)
print 'Test data generated'


Loaded model


Test data generated


In [30]:


x_test = mms.transform(df[data_cols].values)
y1_test=model.predict(x_test)
print y1_test
y_test = np.argmax(y1_test, axis=1)
print y_test

df['is_attributed'] = y_test
results = df[['click_id', 'is_attributed']]
results.to_csv('/Users/bjayaram/Documents/Kaggle/talkingdata/results.csv', index=None)


[[ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]
 ..., 
 [ 1.  0.]
 [ 1.  0.]
 [ 1.  0.]]
[0 0 0 ..., 0 0 0]


In [30]:
print 'done'

done
